In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from scipy.stats import friedmanchisquare
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy.stats import wilcoxon
from itertools import combinations
import openpyxl
import scienceplots

In [2]:
#Load Questionairs (in flight)
# se base foler

# Define the base folder path
#base_folder = r'C:\Users\ejnar\Desktop\Special sync\Special'
base_folder = r'C:\Users\ejnar\OneDrive\DTU\speciale_projekt\Spirometer fra test'
# Define the path to the Excel file
excel_file_path1 = os.path.join(base_folder, 'Full_spyro_output.xlsx')

# Load the Excel file into a DataFrame
df = pd.read_excel(excel_file_path1)

In [3]:
# Specify the columns you want to keep
columns_to_keep = ['SESSION_DATE_TIME','SUBJECT_ID','filter']

# Keep only the specified columns
df = df[columns_to_keep]

In [4]:
# Function to determine Medical_Test based on time
def assign_medical_test(session_time):
    time = session_time.time()
    if pd.Timestamp('09:00').time() <= time <= pd.Timestamp('10:30').time():
        return 1
    elif pd.Timestamp('14:30').time() <= time <= pd.Timestamp('17:00').time():
        return 2
    elif pd.Timestamp('18:00').time() <= time <= pd.Timestamp('21:00').time():
        return 3
    else:
        return None  # You can use 0 or another value for times outside these ranges

# Apply the function to create Medical_Test column
df['Medical_Test'] = df['SESSION_DATE_TIME'].apply(assign_medical_test)

In [5]:
# Assuming df is your DataFrame
df['SESSION_DATE_TIME'] = pd.to_datetime(df['SESSION_DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

# Sort the DataFrame by the SESSION_DATE_TIME column
df = df.sort_values(by='SESSION_DATE_TIME')

#reset the index after sorting
df.reset_index(drop=True, inplace=True)

In [6]:
# Define the base folder where the Vision test data is located
base_folder2 = r'C:\Users\ejnar\OneDrive\DTU\speciale_projekt\Vision\Vision test'

# List of days to exclude
excluded_days = [
    '2024-09-07', '2024-09-08', '2024-09-09', '2024-09-14', '2024-09-15',
    '2024-09-16', '2024-09-21', '2024-09-22', '2024-09-23'
]

# Initialize DataFrames to store the data
df_landoltC = pd.DataFrame(columns=['Acuity Landolt C Score	value'])
reaction_times = []
correct_answers = []
min_values_column_5 = []

# Iterate through all the days
for day in pd.date_range('2024-09-03', '2024-09-27').strftime('%Y-%m-%d'):
    # Skip excluded days
    if day in excluded_days:
        continue
    
    # Iterate through the three medical tests
    for test_num in range(1, 4):
        # Iterate through all 17 test files for each medical test
        for test_file_num in range(1, 18):  # 1 to 17 for each test
            sub_folder = day
            sub_sub_folder = f'Medical test {test_num}'
            
            # Define the file name based
            file_name = f'{day[5:7]}{day[8:10]}MT{test_num}_{test_file_num}.txt'  # Example: 0903MT1_1.txt
            
            # Build the file path
            excel_file_path1 = os.path.join(base_folder2, sub_folder, sub_sub_folder, file_name)
            
            # Check if the file exists
            if os.path.exists(excel_file_path1):
                # Load the file into a DataFrame
                df2 = pd.read_csv(excel_file_path1, sep='\t', encoding='utf-16')
                
                # Extract the specific data from the 14th column
                if len(df2.columns) > 13:
                    landoltC_column_name = df2.columns[13]
                    df_landoltC.at[len(df_landoltC), 'Acuity Landolt C Score	value'] = landoltC_column_name
                
                # Reaction time calculation
                if "." in df2.columns:
                    df2["."] = pd.to_numeric(df2["."], errors='coerce')
                    reaction_time = df2["."].iloc[2:].sum()
                    reaction_times.append(reaction_time)
                
                # Count correct answers
                if "decimalMark" in df2.columns:
                    df2["decimalMark"] = df2["decimalMark"].apply(lambda x: str(x).strip().lower())
                    count_true = df2["decimalMark"].iloc[1:].apply(lambda x: x == "true").sum()
                    correct_answers.append(count_true)
                
                # Minimum value in column
                if "5" in df2.columns and "decimalMark" in df2.columns:
                    df2["5"] = pd.to_numeric(df2["5"], errors="coerce")
                    true_rows = df2[df2["decimalMark"] == "true"]
                    if not true_rows.empty:
                        min_value = true_rows["5"].min()
                        min_values_column_5.append(min_value)
                    else:
                        min_values_column_5.append(None)
            else:
                print(f"File not found: {excel_file_path1}")

# Convert the collected results
df_results = pd.DataFrame({
    'Reaction Time': reaction_times,
    'Correct Answers': correct_answers,
    'Min Correct value': min_values_column_5
})



In [7]:
df = pd.merge(df, df_results, left_index=True, right_index=True, how='outer')
#muligvis fiks senere
df = pd.merge(df, df_landoltC, left_index=True, right_index=True, how='outer')



In [8]:
# Rename the column in the DataFrame
df = df.rename(columns={"Acuity Landolt C Score\tvalue": "Acuity Landolt C Score"})


In [9]:
#exstra filter 
participants_to_keep = [1, 2, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 
                        30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 
                        57, 58, 59, 61, 63, 64, 65, 66, 67, 68, 69]

# Filter the DataFrame
df = df[df['SUBJECT_ID'].isin(participants_to_keep)]

In [10]:
# Assuming your DataFrame is named `df`
df = df[df['filter'] != 'error']


In [11]:
# Ensure SESSION_DATE_TIME is in datetime format
df['SESSION_DATE_TIME'] = pd.to_datetime(df['SESSION_DATE_TIME'])

# Extract day and time as separate columns
df['SESSION_DATE'] = df['SESSION_DATE_TIME'].dt.date  # Extract only the date part
df['SESSION_TIME'] = df['SESSION_DATE_TIME'].dt.time  # Extract only the time part

# Sort by day first, then by time
df = df.sort_values(by=['SESSION_DATE', 'SESSION_TIME'], ascending=True)

# Drop temporary columns
df.drop(columns=['SESSION_DATE', 'SESSION_TIME'], inplace=True)

# Reset index if needed
df.reset_index(drop=True, inplace=True)



In [12]:
# Assuming 'df' is your DataFrame
df['Acuity Landolt C Score'] = pd.to_numeric(df['Acuity Landolt C Score'], errors='coerce')

# Rename the column with the tab character
df.rename(columns={'Acuity Landolt C Score': 'LogMar value'}, inplace=True)


In [13]:
# Convert Reaction Time from ms to s
df['Reaction Time (S)'] = df['Reaction Time'] / 1000


In [14]:
#Rename collums
df.rename(columns={'Reaction Time (S)': 'Completion Time'}, inplace=True)
df.rename(columns={'Correct Answers': 'Correct Answers'}, inplace=True)

In [15]:
#divided into filter
dfHepa7 = df[df['filter'] == 'A']
dfAPSU = df[df['filter'] == 'B']
dfHepa5 = df[df['filter'] == 'C']
dfAPSN = df[df['filter'] == 'D']

In [16]:
#Boxplot creation
def sanitize_filename(name):
    """Replace invalid characters in a filename with underscores."""
    return re.sub(r'[<>:"/\\|?*\t]', '_', name)


def create_combined_boxplots(dataframes, labels, columns_to_plot, name_suffix="", nametag="", y_min=0, y_max=102):
    """Create combined box plots for specified columns in a list of DataFrames."""
    # Get the current directory where the script is placed
    current_directory = os.getcwd()
    output_folder = os.path.join(current_directory, "figur")

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Apply the 'science' style and 'no-latex'
    plt.style.use(['science', 'no-latex'])

    # Define the color palette to match the second code
    colors = ['#77AADD', '#99DDFF', '#44BB99', '#BBCC33']
    sns.set_palette(colors)

    # Assuming all DataFrames have the same 'Medical-Test' values
    medical_tests = dataframes[0]['Medical_Test'].unique()

    # Define custom x-tick labels
    custom_labels = ["MT1", "MT2", "MT3"]

    # Loop through the specified columns in the DataFrames
    for column in columns_to_plot:
        if column in dataframes[0].columns and pd.api.types.is_numeric_dtype(dataframes[0][column]):  # Check if column is numeric
            plt.figure(figsize=(10, 7))  # Match figure size

            # Combine data for seaborn boxplot
            combined_data = pd.concat([
                dataframe.assign(Dataset=labels[i]) 
                for i, dataframe in enumerate(dataframes)
            ])

            # Create the box plot
            sns.boxplot(
                x='Medical_Test', y=column, hue='Dataset', data=combined_data,
                palette=colors, dodge=True
            )

            # Add titles and labels
            plt.title(f"{column} - {nametag}",
                  fontsize=20, pad=15)
            plt.xlabel('Medical Test', fontsize=18)
            plt.ylabel('Number of Correct Responses [-]', fontsize=18)

            # Set y-axis limits
            plt.ylim(y_min, y_max)

            # Set x-ticks and their labels
            plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=16)
            plt.yticks(fontsize=16)

            # Add legend styled like the second code
            plt.legend(
                title="Conditions", title_fontsize=16, fontsize=14,
                loc='center left', bbox_to_anchor=(1, 0.5), frameon=False
            )

            # Add grid lines for y-axis
            plt.grid(visible=True, linestyle='--', linewidth=0.5, alpha=0.75, axis='y')

            # Adjust layout for better spacing
            plt.tight_layout()

            # Save the plot
            plot_filename = f"{sanitize_filename(column)}_{sanitize_filename(name_suffix)}.png"
            plt.savefig(os.path.join(output_folder, plot_filename), bbox_inches='tight')
            plt.close()

In [17]:
#først plot som er raw data 
dataframesRaw = [dfHepa7, dfHepa5,dfAPSN, dfAPSU]  


labels = ['HEPA 7', 'HEPA 5', 'APS N', 'APS U']  # Labels for each DataFrame

output_folder = base_folder

name_suffix = 'All'
nametag = name_suffix
columns_to_plot1 = ['Completion Time']
columns_to_plot2 = ['Percentage Correct']
columns_to_plot3 = ['Correct Answers']
columns_to_plot4 = ['LogMar value']
columns_to_plot5 = ['Min Correct value']





# Create the combined boxplots with a name suffix and custom colors
create_combined_boxplots(dataframesRaw, labels,columns_to_plot1, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=100)
create_combined_boxplots(dataframesRaw, labels,columns_to_plot2, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=100)
create_combined_boxplots(dataframesRaw, labels,columns_to_plot3, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=18)
create_combined_boxplots(dataframesRaw, labels,columns_to_plot4, name_suffix=name_suffix,nametag=nametag, y_min=-1, y_max=1)
create_combined_boxplots(dataframesRaw, labels,columns_to_plot5, name_suffix=name_suffix,nametag=nametag, y_min=-1, y_max=1)



In [18]:
# Define output file path
# freidman willcox by time
output_path = f'Friedman-Wilcoxon-by-Questinair-Vision-{name_suffix}.xlsx'

# DataFrames for each dataset
dataframes = {
    "Hepa 7": dfHepa7,
    "Hepa 5": dfHepa5,
    "APSU": dfAPSU,
    "APSN": dfAPSN
}

# Specify columns not to test
columns_not_to_test = ['SUBJECT_ID', 'Medical_Test','SESSION_DATE_TIME','filter','Reaction Time']

# Alpha significance level
alpha = 0.1

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for dataset_name, df in dataframes.items():
        # Determine columns to test by excluding the specified ones
        columns_to_test = df.columns.difference(columns_not_to_test).tolist()

        # Initialize DataFrames to store result
        friedman_results_df = pd.DataFrame(columns=["Column", "Statistic", "p-value", "Alpha", "Significant"])
        wilcoxon_results_list = []

        # Loop through each column
        for col in columns_to_test:
            # Filter out data for each
            medical_test_groups = [df[df["Medical_Test"] == i][col].dropna().values for i in range(1, 4)]
            
            # Perform the Friedman test
            friedman_stat, friedman_p = friedmanchisquare(*medical_test_groups)
            
            # Determine if the result is significant
            is_significant = 1 if friedman_p < alpha else 0
            
            # Append Friedman test result
            temp_df = pd.DataFrame({
                "Column": [col],
                "Statistic": [friedman_stat],
                "p-value": [friedman_p],
                "Alpha": [alpha],
                "Significant": [is_significant]
            })
            friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)

            # If significant
            if is_significant:
                # Get all pairs of Medical_Test values
                pairs = combinations(range(1, 4), 2)
                for (i, j) in pairs:
                    # Get data for each pair
                    group1 = df[df["Medical_Test"] == i][col].dropna().values
                    group2 = df[df["Medical_Test"] == j][col].dropna().values
                    
                    # Ensure both groups have data for the Wilcoxon test
                    if len(group1) > 0 and len(group2) > 0:
                        wilcox_stat, wilcox_p = wilcoxon(group1, group2)
                        wilcox_significant = 1 if wilcox_p < alpha else 0  
                        
                        # Append results to the Wilcoxon results list
                        wilcoxon_results_list.append({
                            "Column": col,
                            "Pair": f"{i}-{j}",
                            "Statistic": wilcox_stat,
                            "p-value": wilcox_p,
                            "Alpha": alpha,
                            "Significant": wilcox_significant
                        })

        # Save results for the current dataset in separate sheets
        friedman_results_df.to_excel(writer, sheet_name=f"{dataset_name} Friedman Results", index=False)
        
        if wilcoxon_results_list:
            wilcoxon_df = pd.DataFrame(wilcoxon_results_list)
            wilcoxon_df.to_excel(writer, sheet_name=f"{dataset_name} Wilcoxon Results", index=False)

print(f"Friedman and Wilcoxon test results saved to {output_path}.") 

C:\Users\ejnar\AppData\Local\Temp\ipykernel_9552\4019445914.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)
C:\Users\ejnar\AppData\Local\Temp\ipykernel_9552\4019445914.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)
C:\Users\ejnar\AppData\Local\Temp\ipykernel_9552\4019445914.py:48: FutureWarning: The behavior of DataFrame 

Friedman and Wilcoxon test results saved to Friedman-Wilcoxon-by-Questinair-Vision-All.xlsx.


In [19]:
# Define output file path
# freidman willcox by conditions
output_path = f'Friedman-Wilcoxon-by-Filter-Vision-{name_suffix}.xlsx'

# DataFrames for each dataset
dataframes = {
    "Hepa 7": dfHepa7,
    "Hepa 5": dfHepa5,
    "APSU": dfAPSU,
    "APSN": dfAPSN
}


columns_not_to_test = ['SUBJECT_ID', 'Medical_Test', 'SESSION_DATE_TIME', 'filter','Reaction Time']

# Alpha significance level
alpha = 0.1

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    # Loop through each Medical_Test value
    for medical_test in [1, 2, 3]:
        # Create a new DataFrame for Friedman result
        friedman_results_df = pd.DataFrame(columns=["Column", "Statistic", "p-value", "Alpha", "Significant"])
        wilcoxon_results_list = []  

        # Determine columns to test
        columns_to_test = dataframes["Hepa 7"].columns.difference(columns_not_to_test).tolist()

        # Loop through each column
        for col in columns_to_test:
            # Collect data across datasets for the current column and Medical_Test
            data_groups = [
                dataframes[name][dataframes[name]["Medical_Test"] == medical_test][col].dropna().values
                for name in ["Hepa 7", "Hepa 5", "APSU", "APSN"]
            ]

            # Perform the Friedman test
            friedman_stat, friedman_p = friedmanchisquare(*data_groups)

            # Determine if the result is significant
            is_significant = 1 if friedman_p < alpha else 0

            # Append Friedman test results
            friedman_results_df = pd.concat([
                friedman_results_df,
                pd.DataFrame({
                    "Column": [col],
                    "Statistic": [friedman_stat],
                    "p-value": [friedman_p],
                    "Alpha": [alpha],
                    "Significant": [is_significant]
                })
            ], ignore_index=True)

            # If significant
            if is_significant:
                pairs = combinations(["Hepa 7", "Hepa 5", "APSU", "APSN"], 2)
                for (name1, name2) in pairs:
                    group1 = dataframes[name1][dataframes[name1]["Medical_Test"] == medical_test][col].dropna().values
                    group2 = dataframes[name2][dataframes[name2]["Medical_Test"] == medical_test][col].dropna().values

                    # Ensure both groups have data
                    if len(group1) > 0 and len(group2) > 0:
                        wilcox_stat, wilcox_p = wilcoxon(group1, group2)
                        wilcox_significant = 1 if wilcox_p < alpha else 0

                        # Append Wilcoxon results
                        wilcoxon_results_list.append({
                            "Column": col,
                            "Pair": f"{name1} vs {name2}",
                            "Statistic": wilcox_stat,
                            "p-value": wilcox_p,
                            "Alpha": alpha,
                            "Significant": wilcox_significant
                        })

        # Save results 
        friedman_results_df.to_excel(writer, sheet_name=f"Medical {medical_test} Friedman Results", index=False)

        if wilcoxon_results_list:
            wilcoxon_df = pd.DataFrame(wilcoxon_results_list)
            wilcoxon_df.to_excel(writer, sheet_name=f"Medical {medical_test} Wilcoxon Results", index=False)

print(f"Friedman and Wilcoxon test results saved to {output_path}.")


Friedman and Wilcoxon test results saved to Friedman-Wilcoxon-by-Filter-Vision-All.xlsx.


C:\Users\ejnar\AppData\Local\Temp\ipykernel_9552\3540842675.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([
C:\Users\ejnar\AppData\Local\Temp\ipykernel_9552\3540842675.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([
C:\Users\ejnar\AppData\Local\Temp\ipykernel_9552\3540842675.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer

In [20]:
# Define the filename
#Save Dataframe
filename = "Vision_output.xlsx"

# Get the current working directory
current_dir = os.getcwd()
file_path = os.path.join(current_dir, filename)

# Initialize an Excel writer
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # Loop through each dataset and save it to a separate sheet
    for dataset_name, df in dataframes.items():
        # Save each DataFrame to its corresponding sheet
        df.to_excel(writer, sheet_name=dataset_name, index=False)

print(f"DataFrames saved to {file_path}")

DataFrames saved to C:\Users\ejnar\OneDrive\DTU\speciale_projekt\Vision\Vision_output.xlsx
